# 라이브러리 import

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from transformers import BertModel, AutoTokenizer
import torch

# train 데이터 호출

In [ ]:
train = pd.read_csv('data/train.csv')

In [ ]:
train.head()

# text 토큰화

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base', padding=True, max_length=256)

In [ ]:
train['token'] = train['text'].apply(lambda x: torch.tensor([tokenizer.encode(x, padding='max_length', max_length=256)[1:-1]]))

In [ ]:
train.head()

# bert 모델 last hidden state 출력

In [ ]:
model = BertModel.from_pretrained("bert-base-multilingual-uncased")

In [ ]:
train['last_HS'] = train['token'].apply(lambda x: model(x)[0][0][0].unsqueeze(0))

# KMeans by token

In [ ]:
kmeans = KMeans(n_clusters=7, init='k-means++', random_state=42)

In [ ]:
data = torch.stack(train['token'].tolist())
data = torch.flatten(data, start_dim=1)

In [ ]:
labels = kmeans.fit_predict(data)

In [ ]:
train['new_target_tokens'] = labels

In [ ]:
train.head()

In [ ]:
train[train['target'] == 0]['new_target_tokens'].value_counts()

# KMeans by last_HS

In [ ]:
kmeans = KMeans(n_clusters=7, init='k-means++', random_state=42)

In [ ]:
data = torch.stack(train['last_HS'].tolist())
data = torch.flatten(data, start_dim=1)

In [ ]:
labels = kmeans.fit_predict(data)

In [ ]:
train['new_target_last_HS'] = labels

In [ ]:
train[train['target'] == 0]['new_target_last_HS'].value_counts()